In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from transformers import HubertForSequenceClassification, AutoConfig, Wav2Vec2FeatureExtractor
import pandas as pd
import librosa
import numpy as np
from tqdm import tqdm

In [2]:
df = pd.read_csv('../dataset/train.csv')

In [3]:
def convert_labels(label):
    if label == 'fake':
        return 0
    elif label == 'real':
        return 1
    else:
        raise ValueError(f"Unknown label: {label}")

# apply 함수를 사용하여 라벨 변환
df['label'] = df['label'].apply(convert_labels)

In [4]:
# 디바이스 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# 모델 및 특징 추출기 설정
model_name_or_path = 'facebook/hubert-large-ll60k'
config = AutoConfig.from_pretrained(model_name_or_path, num_labels=2)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
sampling_rate = feature_extractor.sampling_rate

/home/work/joono/anaconda3/envs/lcm/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# 음성 파일을 배열로 변환하는 함수
def speech_file_to_array_fn(path):
    audio, _ = librosa.load(path, sr=sampling_rate)
    inputs = feature_extractor(audio, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    return inputs.input_values.squeeze()

# 모델 클래스 정의
class EModel(nn.Module):
    def __init__(self):
        super(EModel, self).__init__()
        self.backbone = HubertForSequenceClassification.from_pretrained(model_name_or_path, config=config)

    def forward(self, x):
        return self.backbone(x).logits

In [6]:
# Collate 함수 정의
def collate_fn(batch):
    signals, labels = zip(*batch)
    max_length = max([signal.size(0) for signal in signals])
    padded_signals = torch.zeros(len(signals), max_length)
    for i, signal in enumerate(signals):
        padded_signals[i, :signal.size(0)] = signal
    labels = torch.tensor(labels)
    return padded_signals, labels

# Mixup 함수 정의 (필요한 경우)
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [7]:
# Trainer 함수 정의
def trainer(model, train_loader, loss_fn, optimizer, epoch):
    model.train()
    train_loss = 0
    step = 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} [Training]"):
        try:
            inputs = inputs.to(device)
            labels = labels.to(device)

            if step % 4 == 0:
                x_batch, y_batch_a, y_batch_b, lam = mixup_data(inputs, labels)
                outputs = model(x_batch)
                loss = mixup_criterion(loss_fn, outputs, y_batch_a.to(device), y_batch_b.to(device), lam)
            else:
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.detach().cpu().item()
            step += 1
        except Exception as e:
            print(f"Error during training: {e}")
    print(f'EPOCH : {epoch} | train_loss : {train_loss/len(train_loader):.4f}')

# Validator 함수 정의 (필요한 경우)
def validator(model, valid_loader, loss_fn, epoch, fold, scheduler):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1} [Validation]"):
            try:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            except Exception as e:
                print(f"Error during validation: {e}")
    
    val_loss /= len(valid_loader)
    val_accuracy = correct / total
    scheduler.step(val_loss)
    
    torch.save(model.state_dict(), f'modelW_3_0.001_{val_accuracy:.6f}_{val_loss:.6f}.pth')
    print(f'Fold {fold+1}, Epoch {epoch + 1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')


In [8]:
from transformers import AdamW

# 모델 로드 (가정: 이미 로드된 상태)
model = EModel().to(device)
loss_fn = nn.CrossEntropyLoss()

# 전체 모델의 파라미터를 고정
for param in model.parameters():
    param.requires_grad = False


for param in model.backbone.classifier.parameters():
    param.requires_grad = True

# 학습 가능한 파라미터만 옵티마이저에 전달
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1)


Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-large-ll60k and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/work/joono/anaconda3/envs/lcm/lib/python3.12/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
'''
# 모델, 손실 함수, 옵티마이저, 스케줄러 정의
model = EModel().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1)
'''

'\n# 모델, 손실 함수, 옵티마이저, 스케줄러 정의\nmodel = EModel().to(device)\nloss_fn = nn.CrossEntropyLoss()\noptimizer = torch.optim.Adam(model.parameters(), lr=1e-5)\nscheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1)\n'

In [10]:
# 데이터셋 클래스 정의
class EMDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        path = os.path.join("..", "dataset", self.df.loc[idx, 'path'])
        if not os.path.exists(path):
            raise FileNotFoundError(f"파일을 찾을 수 없습니다: {path}")
        signal = speech_file_to_array_fn(path)        
        label = self.df.loc[idx, 'label']
        return signal, label

In [11]:
# Stratified K-Fold 설정
k_split = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

In [12]:
# 교차 검증을 통한 모델 훈련
for k, (t_idx, v_idx) in enumerate(k_split.split(df, df['label'])):
    print(f"Fold {k + 1}")
    
    train_df, valid_df = df.loc[t_idx].reset_index(drop=True), df.loc[v_idx].reset_index(drop=True)    

    train_dataset = EMDataset(train_df)
    valid_dataset = EMDataset(valid_df)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn,num_workers=24)
    valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn,num_workers=24)

    for epoch in range(10):
        trainer(model, train_loader, loss_fn, optimizer, epoch)
        validator(model, valid_loader, loss_fn, epoch, k, scheduler)

print("Training complete!")

Fold 1


Epoch 1 [Training]:   0%|          | 0/2772 [00:00<?, ?it/s]/home/work/joono/anaconda3/envs/lcm/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608847532/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
Epoch 1 [Training]: 100%|██████████| 2772/2772 [06:03<00:00,  7.64it/s]


EPOCH : 0 | train_loss : 0.6720


Epoch 1 [Validation]: 100%|██████████| 693/693 [00:42<00:00, 16.43it/s]


Fold 1, Epoch 1, Validation Loss: 0.6642, Validation Accuracy: 0.6111


...

Fold 5, Epoch 14, Validation Loss: 0.8889, Validation Accuracy: 0.5133


Epoch 15 [Training]: 100%|██████████| 2772/2772 [06:19<00:00,  7.31it/s]


EPOCH : 14 | train_loss : 0.4553


Epoch 15 [Validation]: 100%|██████████| 693/693 [00:45<00:00, 15.18it/s]


Fold 5, Epoch 15, Validation Loss: 0.8907, Validation Accuracy: 0.5133
Training complete!


In [13]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        path = os.path.join("..", "dataset", self.df.loc[idx, 'path'])
        if not os.path.exists(path):
            raise FileNotFoundError(f"파일을 찾을 수 없습니다: {path}")     
        signal = speech_file_to_array_fn(path)           
        return signal, -1
        

In [14]:
# 데이터셋 클래스 정의
class EMDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        path = os.path.join("..", "dataset", self.df.loc[idx, 'path'])
        if not os.path.exists(path):
            raise FileNotFoundError(f"파일을 찾을 수 없습니다: {path}")
        signal = speech_file_to_array_fn(path)        
        return signal, -1

In [15]:
test_df = pd.read_csv('../dataset/test.csv', index_col=None)
test_df['path'] = '../dataset/' + test_df['path'].str[1:]
test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=16, collate_fn=collate_fn)

In [16]:
def inference(model, test_loader, device):
    model.to(device)
    model = model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            probs = model(inputs)

            probs = torch.sigmoid(probs)
            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
            
    return predictions

In [17]:
preds = inference(model, test_loader, device)

  0%|          | 0/3125 [00:00<?, ?it/s]/home/work/joono/anaconda3/envs/lcm/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608847532/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 3125/3125 [01:39<00:00, 31.25it/s]


In [18]:
len(preds)

50000

In [19]:
submit = pd.read_csv('../dataset/sample_submission.csv')
submit.iloc[:, 1:] = preds
submit.head()

/tmp/ipykernel_250647/2311052569.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.259104460477829, 0.2502078115940094, 0.2599780857563019, 0.48486849665641785, 0.2734316289424896, 0.3492967188358307, 0.3475615084171295, 0.4910011291503906, 0.38030293583869934, 0.2937047779560089, 0.28343328833580017, 0.23762471973896027, 0.27988407015800476, 0.36559075117111206, 0.26441052556037903, 0.5018996596336365, 0.3510529398918152, 0.22125306725502014, 0.7429962754249573, 0.2559216618537903, 0.2764168083667755, 0.4429420828819275, 0.24574437737464905, 0.23370444774627686, 0.25401005148887634, 0.21976323425769806, 0.34755855798721313, 0.4240126609802246, 0.19843755662441254, 0.28732094168663025, 0.3223840296268463, 0.28076985478401184, 0.5114836096763611, 0.3537074029445648, 0.251844197511673, 0.308030366897583, 0.3023475408554077, 0.32099416851997375, 0.3637951910495758, 0.2550361156463623, 0.4221361577510834, 0.394210

,id,fake,real
0,TEST_00000,0.259104,0.739687
1,TEST_00001,0.250208,0.747689
2,TEST_00002,0.259978,0.740833
3,TEST_00003,0.484868,0.525856
4,TEST_00004,0.273432,0.726768


In [20]:
submit.to_csv('submission/3_0.001_hubert_submit.csv', index=False)